In [22]:
import os
import json
import pandas as pd
import numpy as np

In [23]:
datafile = os.path.join('..', 'dat', '2018')

with open(datafile, 'r') as f:
    record_dicts = []
    for line in f.readlines():
        record = json.loads(line)
        reply_list = record['reply']
        earliest_reply_text = None
        for reply_dict in sorted(reply_list, key=lambda x: x['created_utc']):
            if reply_dict['body'] != '[deleted]' and reply_dict['body'] != '[removed]':
                earliest_reply_text = reply_dict['body']
            if earliest_reply_text:
                break
        if earliest_reply_text:
            record.pop('reply')
            record['reply_text'] = earliest_reply_text
            record_dicts.append(record)

In [24]:
reddit_df = pd.DataFrame(record_dicts)
reddit_df = reddit_df[reddit_df.body != '[deleted]']
reddit_df = reddit_df.astype({'score':np.int64, 'controversiality':np.int64, 'gilded':np.int64, 'created_utc':np.int64})

In [25]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
client = language.LanguageServiceClient()

In [61]:
replies = reddit_df[['body','reply_text']].values
indices = np.arange(len(replies))
np.random.shuffle(indices)
random_idx = indices[:10]

In [63]:
for idx in random_idx:
    op = replies[idx][0]
    post = replies[idx][1]
    lines = post.split('\n')
    for text in lines:
        if text == '':
            continue
        document = types.Document(
            content=text,
            type=enums.Document.Type.PLAIN_TEXT)
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        print("OP:", op)
        print("Text:", text)
        print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))
        print("*"*40)

OP: How much our bridal party could drink when given the news they had an open bar tab. That was a couple thousand extra we didn’t expect to spend.


Actual married life is exactly the same as before we were married since we lived together and shared finances well before marrying. 
Text: I wonder if it would be a good idea to just say it's a discounted bar or something and then at the end reveal it was an open bar.
Sentiment: -0.30000001192092896, 0.30000001192092896
****************************************
OP: excuse me but this is a christian subreddit
Text: But they said no homo
Sentiment: 0.30000001192092896, 0.30000001192092896
****************************************
OP: I don't buy that the inches=pounds thing is real but if you want to add some scientific information to this, for me, one inch was equal to 6.1lbs when I took my starting weight and measurements.
Text: Until I get an accurate scale, I think I'm going to try to do an average of the three numbers I've heard so far. 